In [ ]:
import pandas as pd
import numpy as np
import os
file='./data/voynich.txt'

In [ ]:
#df = pd.read_csv(file,header=None, sep='\t',engine='python',names=['image','description'])
df = pd.read_csv(file,header=None,sep='>',names=['heading','text'])
# df[['image','did']] = df.image.str.split("#",expand=True,)


print(len(df))
df.head(3)

In [ ]:
df['heading'] = df.heading.str.replace('<','')
df['text'] = df.text.str.strip()
df.head(5)

In [ ]:
df[['page','paragraph','line']] = df.heading.str.split(".",expand=True)

In [ ]:
df['page']=df['page'].str.replace('f','')
df['text']=df['text'].str.replace('.',' ')
df['text']=df['text'].str.replace('=',' ')

In [ ]:
df.head(3)

In [ ]:
df.to_csv('data.csv')

In [ ]:
data=df['text']

Creating our own tokenizer

In [ ]:
def get_training_corpus():
    for i in range(0, len(df)):
        yield df.iloc[i : ]["text"]

In [ ]:
import re
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))



In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace((r"[\p{Other}&&[^\n\t\r]]"), ""),
        normalizers.Replace((r"[\s]"), " "),
        normalizers.Lowercase(),
        normalizers.NFD(), normalizers.StripAccents()]
)

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
tokenizer.save('token_voynich.json')

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
df.to_csv('df.csv')

In [ ]:

txt_files_dir='./data/tokenfiles/'

In [ ]:
#Group by paragraph
#datadf=df.groupby(['page','paragraph'])['text'].apply(lambda x: ''.join(x)).reset_index()

#group by page
datadf=df.groupby(['page'])['text'].apply(lambda x: ''.join(x)).reset_index()

In [ ]:
datadf['size']=datadf['text'].str.len()

In [ ]:
datadf.sort_values(by=['page'],inplace=True)


In [ ]:
data=datadf['text']
datadf

In [ ]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
      # Create the filename using the prefix ID
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        # Create the file and write the column text to it
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e:  #catch exceptions(for eg. empty rows)
        print(row, e) 
      i+=1
    # Return the last ID
    return i
# Get the training data

# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
# Print the last ID
print(prefix)
# Get the test data


In [ ]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path


paths = [str(x) for x in Path(".").glob("./data/tokenfiles/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])
#Save the Tokenizer to disk
tokenizer.save_model('./data/')

    


In [ ]:
data

In [ ]:

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True,min_len=3)
        final.append(new)
    return (final)

data_words = gen_words(list(data))

print (data_words[0][0:20])

In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=50,
                                           iterations=150,
                                           alpha="auto")

In [ ]:
lda_model.print_topics()

In [ ]:
qq=data[1]

In [ ]:
datadf.iloc[2,0]

In [ ]:
doctopics=[]
datadf
k=0
for a in data:

    bow = id2word.doc2bow(a.split())
    t = lda_model.get_document_topics(bow,per_word_topics=False,minimum_probability=0.33)
    t.append(datadf.iloc[k,0])
    k=k+1
    doctopics.append(t)
    


In [ ]:
doctopics

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds="mmds", R=20)
vis